<a name="top"></a><img src="./source/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" />

  Before running Spinal HDL code, be sure to load SpinalHDL Libraries  
**Note** : This may be a little slow when the first time load, please wait a moment to download Lib from remote.)

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

## Chapter


In [ ]:
class spRAM(aw: Int, dw: Int, x: Vendor, memName: String = "") extends Component with MemWrap{
  val io = new Bundle{
    val cs, wr = in Bool()
    val addr   = in UInt(aw bits)
    val wdata  = in Bits(dw bits)
    val rdata  = out Bits(dw bits)
  }
  noIoPrefix() 
  val ram = new bf_rfsp(mc, memName)
  ram.Build(this)
}

trait Vendor{
    def rfspMem(mc: MemConfig): MemBlackBox = {new rfsp(mc)}    
    def rf2pMem(mc: MemConfig): MemBlackBox = {new rfsp(mc)}    
    def sfdpMem(mc: MemConfig): MemBlackBox = {new rfsp(mc)}
}

object BigFish extends Vendor{
    override def genMemWrapper(mc: MemConfig): MemBlackBox = {new rfsp(mc)}
}

object SPRD extends Vendor{
    override def genMemWrapper(mc: MemConfig): MemBlackBox = {new rfsp(mc)}
}

object ZTE extends Vendor{
    override def genMemWrapper(mc: MemConfig): MemBlackBox = {new rfsp(mc)}    
    override def genMemWrapper(mc: MemConfig): MemBlackBox = {new rfsp(mc)}
}

In [ ]:
case class MemConfig(dw: Int,
                     depth: Int,
                     maskBitWidth: Int = 0,
                     withScan: Boolean = false,
                     withBist: Boolean = false,
                     withRepaire: Boolean = false){
  def bistaw: Int = 14
  def aw: Int = log2Up(depth)
  def maskName: String = maskBitWidth match {
    case 16 => "wordmask"
    case 8  => "bytemask"
    case 1  => "bitmask"
    case 0  => "nomask"
    case _  => SpinalError("Undefiend BWE type")
  }
  def bweWidth = if(maskBitWidth == 0) 0 else math.ceil(dw.toDouble/maskBitWidth).toInt
  def nonbwe: Boolean = (maskBitWidth == 0)
  def needBWE: Boolean = !nonbwe
  def genBWE(): Bits = in Bits(bweWidth bits)
}

class spRAM(mc: MemConfig, memName: String = "") extends Component with MemWrap{
  val io = new Bundle{
    val cs, wr = in Bool()
    val bwe    = mc.genBWE()
    val addr   = in UInt(mc.aw bits)
    val wdata  = in Bits(mc.dw bits)
    val rdata  = out Bits(mc.dw bits)
  }
  // Map the clk
  // mapCurrentClockDomain(io.clk)
  // Remove io_ prefix
   noIoPrefix()
//  val ram = new ram_1rw(mc, memName)
//  val ram = new vendor.tsmc.mem_1rw(mc, memName)
  val ram = new bf_rfsp(mc, memName)
  ram.Build(this)
}

In [ ]:

class bf_rfsp(mc: MemConfig, memName: String = "") extends MemBlackBox{
  /*Dirty Temporary solution for deadline, TODO*/
 
  this.setDefinitionName("Mwrapper_srsp${mc.depth}x${mc.dw}m16b4w1r")

  val io = new Bundle { iobd =>
    val CLK    = in Bool()
    val A      = in UInt(mc.aw bits)
    val D      = in Bits(mc.dw bits)
    val CEN    = in Bool()
    val WEN    = in Bool()
    val Q      = out Bits(mc.dw bits)

  val cd  = ClockDomain(io.CLK)

  def Build(wrap: spRAM) = {
    wrap.clockDomain.setSyncWith(cd)
    this.io.CLK  := wrap.clockDomain.readClockWire
    this.io.A    := wrap.io.addr
    this.io.CEN  := ~wrap.io.cs
    this.io.WEN  := ~wrap.io.wr
    this.io.D  := wrap.io.wdata
    wrap.io.rdata  := this.io.Q
  }
  noIoPrefix()
//  mapClockDomain(clock = io.CLK)

  val mem = new Mem(Bits(mc.dw bits), mc.depth)
 
  val ca  = new ClockingArea(cd) {
    mem.write(
      address = io.A,
      data = io.D,
      enable = ~io.WEN
    )

    io.Q := mem.readSync(
      address = io.A,
      enable = (!io.CEN && io.WEN)
    )
  }
}
